In [184]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

#for json convertion
import ast

In [185]:
df = pd.read_csv("charging_sessions.csv")

Erstmal Infos sammeln

In [186]:
df.head()

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


In [188]:
fehlende_werte = df.isnull().sum()
fehlende_werte

Unnamed: 0              0
id                      0
connectionTime          0
disconnectTime          0
doneChargingTime     4088
kWhDelivered            0
sessionID               0
siteID                  0
spaceID                 0
stationID               0
timezone                0
userID              17263
userInputs          17263
dtype: int64

Checken ob alle ids und die Spalte unnamed unique Einträge hat

In [189]:
duplicates = df["Unnamed: 0"].unique()
len(duplicates)

15292

In [190]:
duplicates = df["id"].unique()
len(duplicates)

65037

In [191]:
duplicates = df["sessionID"].unique()
len(duplicates)

65037

Die Spalte "Unnamed: 0" geht von 0 bis 15292 und wiederholt sich dann. Also löschen wir die komplett. Außerdem ersetzen wir id mit einer fortlaufenden Integer Nummer von 1-66450.

In [192]:
df.drop(columns="Unnamed: 0", inplace=True)
df["id"] = range(1, len(df) + 1)
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,1,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,2,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,3,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,4,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,5,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


Das Feature sessionID scheint 1413 Duplikate zu haben. Das untersuchen wir genauer.

Die Zeiten werden auf datetime umgeformt, damit wir später einfacher damit rechnen können

In [193]:
df["connectionTime"] = pd.to_datetime(df["connectionTime"])
df["disconnectTime"] = pd.to_datetime(df["disconnectTime"])
df["doneChargingTime"] = pd.to_datetime(df["doneChargingTime"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                66450 non-null  int64              
 1   connectionTime    66450 non-null  datetime64[ns, UTC]
 2   disconnectTime    66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime  62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered      66450 non-null  float64            
 5   sessionID         66450 non-null  object             
 6   siteID            66450 non-null  int64              
 7   spaceID           66450 non-null  object             
 8   stationID         66450 non-null  object             
 9   timezone          66450 non-null  object             
 10  userID            49187 non-null  float64            
 11  userInputs        49187 non-null  object             
dtypes: datetime64[ns, UTC](3), float64(2), int64(2), object(5)
m

Since the time data is in GMT, we will convert it to GMT-8, the time in california.

In [194]:
df["connectionTime"] = df["connectionTime"] - pd.Timedelta(hours=8)
df["disconnectTime"] = df["disconnectTime"] - pd.Timedelta(hours=8)
df["doneChargingTime"] = df["doneChargingTime"] - pd.Timedelta(hours=8)

Checking for invalid data

Possible invalid scenarios are:

    1. connectionTime is after disconnectTime
    2. connectionTime is after doneChargingTime
    3. doneChargingTime is after disconnectTime
    4. The same stationID has two overlapping sessions

In [195]:
disAfterCon = df["connectionTime"] > df["disconnectTime"]
disAfterCon.value_counts()

False    66450
Name: count, dtype: int64

In [196]:
disAfterDone = df["connectionTime"] > df["doneChargingTime"]
disAfterDone.value_counts()

False    66423
True        27
Name: count, dtype: int64

^ TODO: Zeilen untersuchen und evtl löschen

In [197]:
doneAfterDis = df["doneChargingTime"] > df["disconnectTime"]
doneAfterDis.value_counts()

False    61758
True      4692
Name: count, dtype: int64

^ disconnected before charging was done. Not really invalid

now checking for overlapping sessions on the same station

In [198]:
df.sort_values(by=["spaceID", "connectionTime"],inplace=True)
df["overlap"] = ((df["connectionTime"].shift(-1) < df["disconnectTime"]) & (df["spaceID"]==df["spaceID"].shift(-1))).fillna(False)
df["overlap"].value_counts()

overlap
False    65036
True      1414
Name: count, dtype: int64

Seems like 1414 entries are overlapping. We will have a look at a few of such occurences.

In [199]:
overlap_df = df[df["overlap"]]
overlap_df

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,overlap
5584,5585,2019-07-02 05:27:55+00:00,2019-07-02 07:41:50+00:00,2019-07-02 07:42:24+00:00,4.629,1_1_193_825_2019-07-02 13:27:40.720869,1,AG-1F01,1-1-193-825,America/Los_Angeles,2301.0,"[{'WhPerMile': 275, 'kWhRequested': 5.5, 'mile...",True
5627,5628,2019-07-02 08:03:59+00:00,2019-07-02 11:07:33+00:00,2019-07-02 09:38:07+00:00,4.723,1_1_193_825_2019-07-02 16:03:58.713742,1,AG-1F01,1-1-193-825,America/Los_Angeles,1302.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",True
5634,5635,2019-07-02 11:16:01+00:00,2019-07-02 16:08:10+00:00,2019-07-02 16:08:43+00:00,22.883,1_1_193_825_2019-07-02 19:16:00.832510,1,AG-1F01,1-1-193-825,America/Los_Angeles,378.0,"[{'WhPerMile': 250, 'kWhRequested': 22.5, 'mil...",True
5662,5663,2019-07-03 08:34:24+00:00,2019-07-03 12:11:54+00:00,2019-07-03 12:08:35+00:00,11.271,1_1_193_825_2019-07-03 16:34:24.359179,1,AG-1F01,1-1-193-825,America/Los_Angeles,420.0,"[{'WhPerMile': 400, 'kWhRequested': 24.0, 'mil...",True
5670,5671,2019-07-03 18:24:55+00:00,2019-07-03 21:46:55+00:00,2019-07-03 20:58:31+00:00,3.837,1_1_193_825_2019-07-04 02:24:55.416360,1,AG-1F01,1-1-193-825,America/Los_Angeles,3012.0,"[{'WhPerMile': 300, 'kWhRequested': 6.0, 'mile...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6651,6652,2019-07-25 05:41:27+00:00,2019-07-25 15:02:13+00:00,2019-07-25 09:04:48+00:00,5.953,1_1_191_789_2019-07-25 13:41:26.742335,1,AG-4F52,1-1-191-789,America/Los_Angeles,483.0,"[{'WhPerMile': 350, 'kWhRequested': 7.0, 'mile...",True
6734,6735,2019-07-26 06:21:51+00:00,2019-07-26 15:19:14+00:00,2019-07-26 12:33:44+00:00,22.255,1_1_191_789_2019-07-26 14:21:50.885367,1,AG-4F52,1-1-191-789,America/Los_Angeles,1528.0,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'mil...",True
6792,6793,2019-07-29 05:04:38+00:00,2019-07-29 15:40:40+00:00,2019-07-29 07:37:49+00:00,5.893,1_1_191_789_2019-07-29 13:04:38.403533,1,AG-4F52,1-1-191-789,America/Los_Angeles,483.0,"[{'WhPerMile': 350, 'kWhRequested': 7.0, 'mile...",True
6859,6860,2019-07-30 04:41:20+00:00,2019-07-30 15:09:37+00:00,2019-07-30 07:02:41+00:00,5.892,1_1_191_789_2019-07-30 12:41:19.961135,1,AG-4F52,1-1-191-789,America/Los_Angeles,483.0,"[{'WhPerMile': 350, 'kWhRequested': 7.0, 'mile...",True


Somehow, they are not really overlapping and after giving it some thought, since all sessionIDs are unique, it is possible, that some stations can charge two cars at the same time.

New feature ParkDuration to check for outliers

In [200]:
df["parkDuration"] = df["disconnectTime"] - df["connectionTime"]
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,overlap,parkDuration
51322,51323,2020-11-18 15:36:26+00:00,2020-11-18 16:02:37+00:00,NaT,4.816,2_39_81_4550_2020-11-18 23:36:26.012461,2,11900388,2-39-81-4550,America/Los_Angeles,7132.0,"[{'WhPerMile': 274, 'kWhRequested': 8.22, 'mil...",False,0 days 00:26:11
51323,51324,2020-11-18 16:35:54+00:00,2020-11-18 17:31:08+00:00,NaT,10.027,2_39_81_4550_2020-11-19 00:35:53.924922,2,11900388,2-39-81-4550,America/Los_Angeles,4903.0,"[{'WhPerMile': 258, 'kWhRequested': 51.6, 'mil...",False,0 days 00:55:14
51324,51325,2020-11-18 17:34:02+00:00,2020-11-18 18:45:14+00:00,NaT,24.486,2_39_81_4550_2020-11-19 01:33:46.845927,2,11900388,2-39-81-4550,America/Los_Angeles,4903.0,"[{'WhPerMile': 258, 'kWhRequested': 51.6, 'mil...",False,0 days 01:11:12
51327,51328,2020-11-18 19:52:00+00:00,2020-11-18 20:00:50+00:00,NaT,4.788,2_39_81_4550_2020-11-19 03:51:59.755295,2,11900388,2-39-81-4550,America/Los_Angeles,1085.0,"[{'WhPerMile': 283, 'kWhRequested': 56.6, 'mil...",False,0 days 00:08:50
51328,51329,2020-11-18 20:24:11+00:00,2020-11-18 21:07:15+00:00,NaT,30.849,2_39_81_4550_2020-11-19 04:24:10.706432,2,11900388,2-39-81-4550,America/Los_Angeles,9284.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil...",False,0 days 00:43:04


In [201]:
df["parkDuration"].describe()

count                        66450
mean     0 days 06:17:34.578209179
std      0 days 04:51:41.391314114
min                0 days 00:02:04
25%         0 days 02:51:57.750000
50%                0 days 06:11:44
75%         0 days 09:13:31.500000
max               10 days 05:16:09
Name: parkDuration, dtype: object

Check for Outliers

In [202]:
from datetime import timedelta

print("park duration < 5min: ",len(df[df["parkDuration"]<timedelta(minutes=5)]))
print("park duration > 3h: ",len(df[df["parkDuration"]>timedelta(hours=3)]))
print("park duration > 6h: ",len(df[df["parkDuration"]>timedelta(hours=6)]))
print("park duration > 9h: ",len(df[df["parkDuration"]>timedelta(hours=9)]))
print("park duration > 12h: ",len(df[df["parkDuration"]>timedelta(hours=12)]))
print("park duration > 1d: ",len(df[df["parkDuration"]>timedelta(days=1)]))
print("park duration > 2d: ",len(df[df["parkDuration"]>timedelta(days=2)]))

park duration < 5min:  16
park duration > 3h:  49077
park duration > 6h:  33938
park duration > 9h:  18873
park duration > 12h:  2180
park duration > 1d:  279
park duration > 2d:  92


seems like 16 entries were shorter than 5 mins and 279 were longer than 1 day, of which 92 were longer than 2 days

In [203]:
print("park duration > 5d: ",len(df[df["parkDuration"]>timedelta(days=5)]))
print(df["userInputs"].dtype)
df["userInputs"] = df["userInputs"].astype(str)
print(df["userInputs"].dtype)

park duration > 5d:  7
object
object


7 even stayed longer than 5 days.

Convert User Inputs into table columns

Start by only keeping the last userInput Json

In [204]:
df["userInputs"] = df["userInputs"].str.rsplit("{", n=1).str[1]
df["userInputs"] = "[{" + df["userInputs"]

In [205]:
#convert userInputs from string and float to lists

def safe_eval(x):
    try:
        # Check if it's a string and try parsing it
        if isinstance(x, str):
            return ast.literal_eval(x)
        # If it's NaN or None, return an empty list
        elif x is None or (isinstance(x, float) and np.isnan(x)):
            return []
        # If it's already a list, return as is
        elif isinstance(x, list):
            return x
        # If it's some other unexpected object, return as is or log a warning
        else:
            return []  # Default to empty list for unrecognized cases
    except (ValueError, SyntaxError, TypeError):
        # Handle parsing errors safely
        return []

# Apply the safe_eval function to the 'userInputs' column
df['userInputs'] = df['userInputs'].apply(safe_eval)

# Verify the data types after applying the function
print(df['userInputs'].apply(type).unique())

[<class 'list'>]


In [206]:
# Explode and normalize userInputs
exploded_user_inputs = df['userInputs'].explode()

# Normalize the dictionaries into columns
user_inputs_normalized = pd.json_normalize(exploded_user_inputs)

# Restore the original index for merging
user_inputs_normalized['original_index'] = exploded_user_inputs.index

# Merge normalized data back into the original DataFrame
df = df.merge(user_inputs_normalized, left_index=True, right_on='original_index', how='left')

# Drop the auxiliary column
df.drop(columns=['original_index'], inplace=True, errors='ignore')

# Inspect the updated DataFrame
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,...,overlap,parkDuration,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,userID_y
0,51323,2020-11-18 15:36:26+00:00,2020-11-18 16:02:37+00:00,NaT,4.816,2_39_81_4550_2020-11-18 23:36:26.012461,2,11900388,2-39-81-4550,America/Los_Angeles,...,False,0 days 00:26:11,274.0,8.22,30.0,480.0,"Wed, 18 Nov 2020 23:36:42 GMT",True,"Thu, 19 Nov 2020 07:36:26 GMT",7132.0
1,51324,2020-11-18 16:35:54+00:00,2020-11-18 17:31:08+00:00,NaT,10.027,2_39_81_4550_2020-11-19 00:35:53.924922,2,11900388,2-39-81-4550,America/Los_Angeles,...,False,0 days 00:55:14,258.0,51.60,200.0,576.0,"Thu, 19 Nov 2020 00:36:25 GMT",True,"Thu, 19 Nov 2020 10:11:54 GMT",4903.0
2,51325,2020-11-18 17:34:02+00:00,2020-11-18 18:45:14+00:00,NaT,24.486,2_39_81_4550_2020-11-19 01:33:46.845927,2,11900388,2-39-81-4550,America/Los_Angeles,...,False,0 days 01:11:12,258.0,51.60,200.0,576.0,"Thu, 19 Nov 2020 01:34:02 GMT",True,"Thu, 19 Nov 2020 11:10:02 GMT",4903.0
3,51328,2020-11-18 19:52:00+00:00,2020-11-18 20:00:50+00:00,NaT,4.788,2_39_81_4550_2020-11-19 03:51:59.755295,2,11900388,2-39-81-4550,America/Los_Angeles,...,False,0 days 00:08:50,283.0,56.60,200.0,589.0,"Thu, 19 Nov 2020 03:52:50 GMT",True,"Thu, 19 Nov 2020 13:41:00 GMT",1085.0
4,51329,2020-11-18 20:24:11+00:00,2020-11-18 21:07:15+00:00,NaT,30.849,2_39_81_4550_2020-11-19 04:24:10.706432,2,11900388,2-39-81-4550,America/Los_Angeles,...,False,0 days 00:43:04,400.0,40.00,100.0,30.0,"Thu, 19 Nov 2020 04:24:50 GMT",True,"Thu, 19 Nov 2020 04:54:11 GMT",9284.0


We remove temporary and unnecessary columns for the next tasks.

In [207]:
df = df.drop("sessionID", axis=1)
df = df.drop("timezone", axis=1)
df = df.drop("overlap", axis=1)
df = df.drop("modifiedAt", axis=1)
df = df.drop("userInputs", axis=1)
df = df.drop("userID_y", axis=1)

In [208]:
df

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID_x,parkDuration,WhPerMile,kWhRequested,milesRequested,minutesAvailable,paymentRequired,requestedDeparture
0,51323,2020-11-18 15:36:26+00:00,2020-11-18 16:02:37+00:00,NaT,4.816,2,11900388,2-39-81-4550,7132.0,0 days 00:26:11,274.0,8.22,30.0,480.0,True,"Thu, 19 Nov 2020 07:36:26 GMT"
1,51324,2020-11-18 16:35:54+00:00,2020-11-18 17:31:08+00:00,NaT,10.027,2,11900388,2-39-81-4550,4903.0,0 days 00:55:14,258.0,51.60,200.0,576.0,True,"Thu, 19 Nov 2020 10:11:54 GMT"
2,51325,2020-11-18 17:34:02+00:00,2020-11-18 18:45:14+00:00,NaT,24.486,2,11900388,2-39-81-4550,4903.0,0 days 01:11:12,258.0,51.60,200.0,576.0,True,"Thu, 19 Nov 2020 11:10:02 GMT"
3,51328,2020-11-18 19:52:00+00:00,2020-11-18 20:00:50+00:00,NaT,4.788,2,11900388,2-39-81-4550,1085.0,0 days 00:08:50,283.0,56.60,200.0,589.0,True,"Thu, 19 Nov 2020 13:41:00 GMT"
4,51329,2020-11-18 20:24:11+00:00,2020-11-18 21:07:15+00:00,NaT,30.849,2,11900388,2-39-81-4550,9284.0,0 days 00:43:04,400.0,40.00,100.0,30.0,True,"Thu, 19 Nov 2020 04:54:11 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,24487,2019-05-13 11:43:12+00:00,2019-05-13 13:54:33+00:00,2019-05-13 12:46:37+00:00,2.308,2,CA-513,2-39-139-567,560.0,0 days 02:11:21,273.0,5.46,20.0,133.0,True,"Mon, 13 May 2019 21:56:12 GMT"
66446,25429,2019-06-13 08:53:42+00:00,2019-06-13 09:35:12+00:00,2019-06-13 09:24:57+00:00,0.908,2,CA-513,2-39-139-567,NaN,0 days 00:41:30,NaN,NaN,NaN,NaN,NaN,NaN
66447,28030,2019-09-16 05:55:08+00:00,2019-09-16 07:34:27+00:00,2019-09-16 06:25:44+00:00,0.893,2,CA-513,2-39-139-567,NaN,0 days 01:39:19,NaN,NaN,NaN,NaN,NaN,NaN
66448,29515,2019-11-06 07:28:18+00:00,2019-11-06 08:41:56+00:00,2019-11-06 07:58:42+00:00,0.900,2,CA-513,2-39-139-567,NaN,0 days 01:13:38,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   id                  66450 non-null  int64              
 1   connectionTime      66450 non-null  datetime64[ns, UTC]
 2   disconnectTime      66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime    62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered        66450 non-null  float64            
 5   siteID              66450 non-null  int64              
 6   spaceID             66450 non-null  object             
 7   stationID           66450 non-null  object             
 8   userID_x            49187 non-null  float64            
 9   parkDuration        66450 non-null  timedelta64[ns]    
 10  WhPerMile           49187 non-null  float64            
 11  kWhRequested        49187 non-null  float64            
 12  milesRequested      49187 non-nu

We create one csv with all of the cleaned data and one with all the doneChargingTime Null Values removed.

In [210]:
df.to_csv("cleanChargingDataFull.csv")
df.dropna(subset=["doneChargingTime"], inplace=True)
df.to_csv("cleanChargingDataNoNull.csv")